# Test task

In [1]:
# import related libraries

import pandas as pd
import numpy as np

#-----------------------------------
import warnings
warnings.filterwarnings("ignore")

In [2]:
# upload data from a relative path
data = pd.read_csv(r"int20h-ds-test-dataset.csv")
data.head() 

,userid,user_state,event_name,event_attributes,event_created_date,event_platform,device_manufacture,device_model
0,c95c777785faec8dd910d019d7278ebe,CA,Add Vehicle Success,"{""Make"":""Dodge"",""Model"":""Caravan"",""Color"":""Whi...",2022-01-16 17:03:04,android,samsung,SM-N975U
1,c95c777785faec8dd910d019d7278ebe,CA,Add Vehicle Break,{},2022-01-16 17:07:47,android,samsung,SM-N975U
2,f344be2d9a042b7444f3cc5279e38ef1,FL,Calculator View,{},2022-01-16 17:16:25,android,samsung,SM-G973U1
3,c95c777785faec8dd910d019d7278ebe,CA,Add Payment Method Success,"{""Payment Method"":""Credit"",""Tokenized Pay"":""""}",2022-01-16 17:24:22,android,samsung,SM-N975U
4,e331ed81422d8fba55520a43a872e701,IL,Sign Up Success,"{""Method"":""Apple""}",2022-01-16 17:34:51,ios,Apple,"iPhone12,1"


## Data preprocessing

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23357 entries, 0 to 23356
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   userid              23357 non-null  object
 1   user_state          23286 non-null  object
 2   event_name          23357 non-null  object
 3   event_attributes    23357 non-null  object
 4   event_created_date  23357 non-null  object
 5   event_platform      23357 non-null  object
 6   device_manufacture  15847 non-null  object
 7   device_model        15847 non-null  object
dtypes: object(8)
memory usage: 1.4+ MB


### Dropping duplicates

As we can see event_created_date has Dtype object and using reference to pandas docs [drop duplicates method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html), we can just delete repeating rows

In [4]:
data.drop_duplicates(inplace=True)

### Filling NA values

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22039 entries, 0 to 23356
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   userid              22039 non-null  object
 1   user_state          21968 non-null  object
 2   event_name          22039 non-null  object
 3   event_attributes    22039 non-null  object
 4   event_created_date  22039 non-null  object
 5   event_platform      22039 non-null  object
 6   device_manufacture  15809 non-null  object
 7   device_model        15809 non-null  object
dtypes: object(8)
memory usage: 1.5+ MB


We can see that we have missing values into columns user_state, device_manufacture and device_model. By the way, the number of missing values in device_manufacture, device_model is the same. So we can check a theory about missing these values in some specific cases

In [6]:
column_name = 'device_manufacture'
data[data[column_name].isna()]

,userid,user_state,event_name,event_attributes,event_created_date,event_platform,device_manufacture,device_model
23,c664cc762bdcb88230b4b4f07e3b7d2e,CA,Transaction Refund,"{""Id"":""708ee8f2012042b121be7dcdd6df256a"",""Amou...",2022-01-16 21:41:05,outofband,NaN,NaN
34,017f4557810b3ba165d2959c17b916b6,TX,Subscription Premium,"{""Start Date"":""2022-01-16"",""Renewal Date"":""202...",2022-01-17 00:04:58,outofband,NaN,NaN
72,0d51394c39dbf15021ccd09155ee6257,FL,Order,"{""Premium Membership"":""True""}",2022-01-15 09:23:48,outofband,NaN,NaN
74,0d51394c39dbf15021ccd09155ee6257,FL,Order,"{""Premium Membership"":""True""}",2022-01-15 10:16:12,outofband,NaN,NaN
85,c67290d72ec2974aba0f3975c1773405,FL,Subscription Premium,"{""Start Date"":""2022-01-16"",""Renewal Date"":""202...",2022-01-17 01:31:29,outofband,NaN,NaN
...,...,...,...,...,...,...,...,...
23337,189adece0dd21eb905f1bfcc09298ecd,CA,Order,"{""Premium Membership"":""True""}",2022-04-16 10:56:36,outofband,NaN,NaN
23338,81d7b7d89e06767d80a1aba25e4e3609,CA,Order,"{""Premium Membership"":""True""}",2022-04-16 10:57:42,outofband,NaN,NaN
23345,2c597755067d64bf12050cbd7b62c30f,MD,Subscription Premium Cancel,{},2022-04-16 14:15:24,outofband,NaN,NaN
23347,d545e0069a53fcc868e01726663cd449,CA,Subscription Premium Cancel,{},2022-04-16 14:50:13,outofband,NaN,NaN


As we see, we have 6230 cases, and they has one common thing: event_platform is outofband. Let's check the situation where this condition is true

In [7]:
data.query("event_platform == 'outofband'")

,userid,user_state,event_name,event_attributes,event_created_date,event_platform,device_manufacture,device_model
23,c664cc762bdcb88230b4b4f07e3b7d2e,CA,Transaction Refund,"{""Id"":""708ee8f2012042b121be7dcdd6df256a"",""Amou...",2022-01-16 21:41:05,outofband,NaN,NaN
34,017f4557810b3ba165d2959c17b916b6,TX,Subscription Premium,"{""Start Date"":""2022-01-16"",""Renewal Date"":""202...",2022-01-17 00:04:58,outofband,NaN,NaN
72,0d51394c39dbf15021ccd09155ee6257,FL,Order,"{""Premium Membership"":""True""}",2022-01-15 09:23:48,outofband,NaN,NaN
74,0d51394c39dbf15021ccd09155ee6257,FL,Order,"{""Premium Membership"":""True""}",2022-01-15 10:16:12,outofband,NaN,NaN
85,c67290d72ec2974aba0f3975c1773405,FL,Subscription Premium,"{""Start Date"":""2022-01-16"",""Renewal Date"":""202...",2022-01-17 01:31:29,outofband,NaN,NaN
...,...,...,...,...,...,...,...,...
23337,189adece0dd21eb905f1bfcc09298ecd,CA,Order,"{""Premium Membership"":""True""}",2022-04-16 10:56:36,outofband,NaN,NaN
23338,81d7b7d89e06767d80a1aba25e4e3609,CA,Order,"{""Premium Membership"":""True""}",2022-04-16 10:57:42,outofband,NaN,NaN
23345,2c597755067d64bf12050cbd7b62c30f,MD,Subscription Premium Cancel,{},2022-04-16 14:15:24,outofband,NaN,NaN
23347,d545e0069a53fcc868e01726663cd449,CA,Subscription Premium Cancel,{},2022-04-16 14:50:13,outofband,NaN,NaN


Here we also have 6230 rows with the same structure as in dataset upper it, so let's fill this na values with outofband, so we can understand the origin of values in that columns

In [8]:
data.loc[data.query("event_platform == 'outofband'").index] = \
data.loc[data.query("event_platform == 'outofband'").index].fillna('outofband')

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22039 entries, 0 to 23356
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   userid              22039 non-null  object
 1   user_state          21968 non-null  object
 2   event_name          22039 non-null  object
 3   event_attributes    22039 non-null  object
 4   event_created_date  22039 non-null  object
 5   event_platform      22039 non-null  object
 6   device_manufacture  22039 non-null  object
 7   device_model        22039 non-null  object
dtypes: object(8)
memory usage: 2.0+ MB


We can see that we didn't trigger any missing value user_state and filled values only in device_manufacture and device_model

Now we can check rows with missing user_state value

In [10]:
column_name = 'user_state' 
data[data[column_name].isna()][:50]

,userid,user_state,event_name,event_attributes,event_created_date,event_platform,device_manufacture,device_model
20,c5cf3fc44b606d4614a834a5306e9616,NaN,Sign Up Success,"{""Method"":""Email""}",2022-01-16 21:01:55,android,TCL,9032Z
93,ed09ebeba14600a9ff4defc64e32503c,NaN,Sign Up Success,"{""Method"":""Email""}",2022-01-17 04:40:32,ios,Apple,"iPhone11,6"
121,9182e94800ca0f8e9094ea3b8f5b169c,NaN,Sign Up Success,"{""Method"":""Google""}",2022-01-15 14:46:05,android,samsung,SM-A426U
148,838db409e7565bfa896beff8bf7fe023,NaN,Sign Up Success,"{""Method"":""Email""}",2022-01-17 15:43:30,android,motorola,moto g stylus 5G
150,838db409e7565bfa896beff8bf7fe023,NaN,Email Confirmation Success,{},2022-01-17 15:45:11,android,motorola,moto g stylus 5G
151,838db409e7565bfa896beff8bf7fe023,NaN,Add Vehicle Success,"{""Make"":""Toyota"",""Model"":""Tacoma"",""Color"":""Whi...",2022-01-17 15:49:17,android,motorola,moto g stylus 5G
975,54621802fbf69592d1972e7fb7ad19ca,NaN,Sign Up Success,"{""Method"":""Email""}",2022-01-22 12:01:56,ios,Apple,"iPhone11,2"
3308,2971d0f3a5edd728ac036117268e4bf5,NaN,Sign Up Success,"{""Method"":""Google""}",2022-02-09 21:35:24,ios,Apple,"iPhone11,8"
4522,7b11702f547fe296f986cfd01c328158,NaN,Sign Up Success,"{""Method"":""Apple""}",2022-02-11 23:52:03,ios,Apple,"iPhone14,3"
4699,7e1beebdff05874880acc85d7b2389da,NaN,Sign Up Success,"{""Method"":""Facebook""}",2022-02-12 21:00:02,ios,Apple,"iPhone13,4"


We don't see a specific behaviour here, so let's do it in other way

In [11]:
# count amount of unique user_state values

data.user_state.value_counts()

CA    8371
TX    5997
FL    1873
IL    1120
NY     931
NJ     824
VA     636
PA     234
MD     230
AZ     229
MA     217
OK     117
IN     103
NC      97
WI      97
DE      78
MI      71
MO      64
CT      56
AR      55
CO      51
OH      43
NH      42
NV      42
WV      31
GA      28
IA      27
SC      27
NM      26
DC      26
UT      26
AL      26
TN      25
KY      23
MN      23
OR      21
ME      13
KS      13
VT      13
MT       9
RI       7
ID       5
NE       5
WA       4
HI       3
SD       3
AK       2
LA       2
MS       2
Name: user_state, dtype: int64

We will leave only 20 states with the most amount of operations, for left values we create a special class 'other'

In [12]:
top_states = data.user_state.value_counts()[:20].keys().to_list()
data.user_state = data.user_state.apply(lambda x: x if x in top_states else 'other')

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22039 entries, 0 to 23356
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   userid              22039 non-null  object
 1   user_state          22039 non-null  object
 2   event_name          22039 non-null  object
 3   event_attributes    22039 non-null  object
 4   event_created_date  22039 non-null  object
 5   event_platform      22039 non-null  object
 6   device_manufacture  22039 non-null  object
 7   device_model        22039 non-null  object
dtypes: object(8)
memory usage: 2.0+ MB


Now we have no missing values

#### Processing sample outliers

In [14]:
data.device_model.value_counts()[:50]

outofband             6230
iPhone12,1            1489
iPhone13,4            1386
iPhone14,3             909
iPhone11,8             671
iPhone13,2             645
iPhone12,5             615
iPhone13,3             588
iPhone14,5             481
iPhone14,2             443
iPhone12,8             419
iPhone12,3             361
SM-G998U               281
iPhone13,1             261
SM-G991U               259
SM-G970U               243
iPhone11,6             231
SM-G781U               211
SM-A326U               208
SM-G975U               163
iPhone10,2             154
SM-A205U               153
SM-A115U               148
SM-A716U               141
SM-N986U               141
SM-G965U               135
iPhone11,2             132
moto g play (2021)     117
iPhone10,5             116
SM-N960U               111
SM-F926U               109
iPhone10,1             100
SM-G960U                99
SM-N975U1               98
SM-G996U                97
SM-A125U                94
Pixel 5                 92
S

We can see that we have a lot of 'versions' of Iphone 11, 12 etc, so we can just combine that values in one unique. Also we will do this process to Motorola phones, iPads, Pixel and [Samsung](https://en.wikipedia.org/wiki/Samsung_Galaxy#Release_history)

In [15]:
for iphone_number in range(7, 15):
    data.device_model.loc[data['device_model'].apply(lambda x: f"iphone{iphone_number}" in str(x).lower())] = \
    'iPhone' + str(iphone_number)
data.device_model.loc[data['device_model'].apply(lambda x: (str(x).lower()).startswith('sm'))] = data['device_model'].str[:5]
data.device_model.loc[data['device_model'].apply(lambda x: (str(x).lower()).startswith('pixel'))] = data['device_model'].str[:7]
data.device_model.loc[data['device_model'].apply(lambda x: "ipad" in str(x).lower())] = 'iPad'
data.device_model.loc[data['device_model'].apply(lambda x: "moto" in str(x).lower())] = 'moto'

Now let's view changed list

In [16]:
data.device_model.value_counts()[:50]

outofband          6230
iPhone12           2884
iPhone13           2880
iPhone14           1900
SM-G9              1897
iPhone11           1043
SM-N9               745
iPhone10            525
moto                505
SM-G7               333
SM-A1               313
SM-A3               233
iPhone9             210
SM-A2               173
SM-A5               151
SM-A7               148
SM-S9               124
SM-A0               123
iPhone8             119
SM-F9               118
Pixel 5             113
Nokia X100           89
Pixel 6              83
Pixel 3              82
REVVL V+ 5G          81
LM-Q730              80
Pixel 4              74
DE2118               61
SM-F7                60
LM-V600              60
LM-K300              57
ONEPLUS A6013        32
LM-V350              29
SM-J7                27
iPad                 25
LM-K500              24
SM-G8                20
LM-V405              17
BE2025               15
TMRVL4G              14
Nokia C2 Tennen      13
SM-S2           

We can mark values with amount < 25 by the name of device_manufacture

In [17]:
top_model = data.device_model.value_counts()[:35].keys().to_list()
data.device_model = data.device_model. \
apply(lambda x: x if x in top_model else data.loc[data.device_model == x].iloc[0].device_manufacture)

Let's see all processed data now

In [18]:
data.describe()

,userid,user_state,event_name,event_attributes,event_created_date,event_platform,device_manufacture,device_model
count,22039,22039,22039,22039,22039,22039,22039,22039
unique,3540,21,23,2309,20452,3,23,55
top,627f50253b42607513a1c93bb68201ad,CA,Order,{},2022-04-10 06:26:47,ios,Apple,outofband
freq,320,8371,4845,4668,6,9586,9586,6230


#### Time series work

In [19]:
# converting event_created_date to pd.datetime object
data.event_created_date = pd.to_datetime(data.event_created_date)

In [20]:
# adding column month with month of operation
data["month"] = data.event_created_date.dt.month

In [21]:
data

,userid,user_state,event_name,event_attributes,event_created_date,event_platform,device_manufacture,device_model,month
0,c95c777785faec8dd910d019d7278ebe,CA,Add Vehicle Success,"{""Make"":""Dodge"",""Model"":""Caravan"",""Color"":""Whi...",2022-01-16 17:03:04,android,samsung,SM-N9,1
1,c95c777785faec8dd910d019d7278ebe,CA,Add Vehicle Break,{},2022-01-16 17:07:47,android,samsung,SM-N9,1
2,f344be2d9a042b7444f3cc5279e38ef1,FL,Calculator View,{},2022-01-16 17:16:25,android,samsung,SM-G9,1
3,c95c777785faec8dd910d019d7278ebe,CA,Add Payment Method Success,"{""Payment Method"":""Credit"",""Tokenized Pay"":""""}",2022-01-16 17:24:22,android,samsung,SM-N9,1
4,e331ed81422d8fba55520a43a872e701,IL,Sign Up Success,"{""Method"":""Apple""}",2022-01-16 17:34:51,ios,Apple,iPhone12,1
...,...,...,...,...,...,...,...,...,...
23352,679eba26c4e75e0afb178360becfa21b,CA,Add Payment Method Success,"{""Payment Method"":""Credit"",""Tokenized Pay"":"""",...",2022-04-16 20:49:24,android,Google,Pixel 3,4
23353,679eba26c4e75e0afb178360becfa21b,CA,Account Setup Profile Skip,"{""Screen"":""Address""}",2022-04-16 20:50:05,android,Google,Pixel 3,4
23354,679eba26c4e75e0afb178360becfa21b,CA,Account Setup Profile Skip,"{""Screen"":""Phone Number""}",2022-04-16 20:50:10,android,Google,Pixel 3,4
23355,679eba26c4e75e0afb178360becfa21b,CA,Chat Conversation Opened,"{""From"":""Dashboard"",""Transaction type"":""""}",2022-04-16 20:50:31,android,Google,Pixel 3,4


In [23]:
# export data
data.to_csv('processed_data.csv', encoding='utf-8', index=False)

### Interim findings
We processed data deleting duplicate values, filling missing values, adding some parameters. We have familiarized ourselves with the dataset and are now ready to work with it in the future

[See notebook Vizualization_plus_attributes]